Multi-Target DMRG ITensor Version: With no U(1) symmetry.

In [1]:
using ITensors
using KrylovKit
using Einsum
using TensorOperations
const nexcit = 3
const maxdim = 20
const cutoff = 0
const virind = Index(nexcit;tags="v")
function get_phi0(psi,b)
    T = psi[b]*psi[b+1]
    s1,s2 =  findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        A = array(T,s1,s2,l[1],l[2],virind)
        phi0 = copy(A[:,:,:,:,1])
        return ITensor(phi0,s1,s2,l[1],l[2])
    else
        A = array(T,s1,s2,l[1],virind)
        phi0 = copy(A[:,:,:,1])
        return ITensor(phi0,s1,s2,l[1])
    end
end
function get_phi_tensor(phi_list,psi,b)
    T = psi[b]*psi[b+1]
    s1,s2 =  findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        AT = array(T,s1,s2,l[1],l[2],virind)
        for n = 1:nexcit
            AT[:,:,:,:,n] = array(phi_list[n],s1,s2,l[1],l[2])
        end
        return ITensor(AT,s1,s2,l[1],l[2],virind)
    else
        AT = array(T,s1,s2,l[1],virind)
        for n = 1:nexcit
            AT[:,:,:,n] = array(phi_list[n],s1,s2,l[1])
        end
        return ITensor(AT,s1,s2,l[1],virind)
    end
end
function Tensor_inner(T1,T2)
    if length(size(T1)) == 2
        @einsum T[m,n] := T1[i,m]*T2[i,n]
        return tr(T)
    else
        @einsum T[m,n] := T1[i,j,m]*T2[i,j,n]
        return tr(T)
    end
end
function eigenvector_normalize(T)
    s = findinds(T;tags="Site")
    l = findinds(T;tags="Link")
    if length(l) == 2
        A = array(T,s[1],l[1],l[2],virind)
        for n = 1:nexcit
            for m = 1:n-1
                A[:,:,:,n] = A[:,:,:,n]-Tensor_inner(A[:,:,:,n],A[:,:,:,m])*A[:,:,:,m]
            end
            A[:,:,:,n] = A[:,:,:,n]/sqrt(Tensor_inner(A[:,:,:,n],A[:,:,:,n]))
        end
        return ITensor(A/sqrt(nexcit),s[1],l[1],l[2],virind)
    else
        A = array(T,s[1],l[1],virind)
        for n = 1:nexcit
            for m = 1:n-1
                A[:,:,n] = A[:,:,n]-Tensor_inner(A[:,:,n],A[:,:,m])*A[:,:,m]
            end
            A[:,:,n] = A[:,:,n]/sqrt(Tensor_inner(A[:,:,n],A[:,:,n]))
        end
        return ITensor(A/sqrt(nexcit),s[1],l[1],virind)
    end
end
function multi_replacebond!(psi,b,phi,ha)
    if ha == 1
        s =  findinds(psi[b];tags="Site")
        if b == 1
            indsb = s
        else
            indsb = (s,linkind(psi,b-1))
        end
        L,R,spec = ITensors.factorize(phi,indsb;which_decomp="svd",maxdim,cutoff,ortho="left",tags=tags(linkind(psi,b)))
        psi[b] = L
        R = eigenvector_normalize(R)
        psi[b+1] = R
    elseif ha == 2
        s =  findinds(psi[b];tags="Site")
        if b == 1
            indsb = (s,virind)
        else
            indsb = (s,linkind(psi,b-1),virind)
        end
        L,R,spec = ITensors.factorize(phi,indsb;which_decomp="svd",maxdim,cutoff,ortho="right",tags=tags(linkind(psi,b)))
        L = eigenvector_normalize(L)
        psi[b] = L
        psi[b+1] = R
    end
    if ha == 1
        ITensors.leftlim(psi) == b - 1 && ITensors.setleftlim!(psi, ITensors.leftlim(psi) + 1)
        ITensors.rightlim(psi) == b + 1 && ITensors.setrightlim!(M, ITensors.rightlim(psi) + 1)
        psi[b + 1] ./= norm(psi[b + 1])
    elseif ha == 2
        ITensors.leftlim(psi) == b && ITensors.setleftlim!(psi, ITensors.leftlim(psi) - 1)
        ITensors.rightlim(psi) == b + 2 && ITensors.setrightlim!(psi, ITensors.rightlim(psi) - 1)
        psi[b] ./= norm(psi[b])
    end
    return spec
end
function HamiltonianMultiply_matrix(P,phi0)
    itensor_map = Union{ITensor,ITensors.OneITensor}[lproj(P)]
    append!(itensor_map, P.H[ITensors.site_range(P)])
    Hv = rproj(P)
    for it in itensor_map
        Hv *= it
    end
    s = inds(phi0)
    d = dims(phi0)
    if length(s) == 4
        Mat = Array(Hv,s[1],s[2],s[3],s[4],s[1]',s[2]',s[3]',s[4]')
        TT = reshape(Mat,(d[1]*d[2]*d[3]*d[4],d[1]*d[2]*d[3]*d[4]))
        e,v = eigen(TT)
        e = e[1:nexcit]
        vec = Vector{Any}(undef, nexcit)
        for n = 1:nexcit
            vec[n] = ITensor(reshape(v[:,n],(d[1],d[2],d[3],d[4])),s[1],s[2],s[3],s[4])
        end
    else
        Mat = Array(Hv,s[1],s[2],s[3],s[1]',s[2]',s[3]')
        TT = reshape(Mat,(d[1]*d[2]*d[3],d[1]*d[2]*d[3]))
        e,v = eigen(TT)
        @show length(e)
        e = e[1:nexcit]
        vec = Vector{Any}(undef, nexcit)
        for n = 1:nexcit
            vec[n] = ITensor(reshape(v[:,n],(d[1],d[2],d[3])),s[1],s[2],s[3])
        end
    end
    return e,vec
end
let
    #define MPO and initial MPS state
    N = 100
    sites = siteinds("S=1/2",N)
    os = OpSum()
    for j in 1:(N-1)
        os .+= 0.5,"S+",j,"S-",j+1
        os .+= 0.5,"S-",j,"S+",j+1
        os .+= "Sz",j,"Sz",j+1
    end
    H = MPO(os,sites)
    state = [isodd(n) ? "Up" : "Dn" for n in 1:N]
    psi0 = MPS(sites,state)
    #set DMRG parameters
    nsweep = 4
    #do DMRG calculation
    PH = ProjMPO(H) #ProjMPO is a type of ITensor
    psi = copy(psi0)
    N = length(psi)
    if !isortho(psi) || orthoCenter(psi)!=1
        psi=ITensors.orthogonalize!(PH,psi,1) # set the orthogonal center at the first site
    end
    slinds = inds(psi[1])
    a1 = array(psi[1],slinds[1],slinds[2])
    a2 = zeros(ComplexF64,(dim(slinds[1]),dim(slinds[2]),nexcit))
    for n = 1:nexcit
        a2[:,:,n] = a1[:,:]
    end
    psi[1] = ITensor(a2,slinds[1],slinds[2],virind)
    PH = position!(PH,psi,1) #Effective Hamiltonian for the first-two sites
    energy = 0.0
    for sw in 1:nsweep
        for (b,ha) in sweepnext(N)
            PH = position!(PH,psi,b) #Effective Hamiltonian for the two sites connected by the bond we want to optimaize
            phi0 = get_phi0(psi,b)
            vals,vecs = eigsolve(PH,phi0,nexcit,:SR;ishermitian=true,eager=true) #get the ground eigenvalue and the ground eigenstate of the coarsed site
            if length(vals) < nexcit
                vals,vecs = HamiltonianMultiply_matrix(PH,phi0)
            end
            energy = copy(vals[1:nexcit])
            phi_list = copy(vecs[1:nexcit])
            phi = get_phi_tensor(phi_list,psi,b)
            spec = multi_replacebond!(psi,b,phi,ha)
            println("site $b : energy = $energy linkdim = $(linkdim(psi,b)) trunc = $(spec.truncerr)")
        end
        println("After sweep $sw energy=$energy")
    end
    psi_list = Vector{Any}(undef, nexcit)
    for n = 1:nexcit
        psin = copy(psi)
        s = findinds(psi[1];tags="Site")
        l = findinds(psi[1];tags="Link")
        A = array(psi[1],s[1],l[1],virind)
        an = copy(A[:,:,n])
        Tn = ITensor(an,s[1],l[1])
        psin[1] = Tn
        psi_list[n] = psin
        println("The $(n-1)th excited state energy is: $(inner(psi_list[n]',H,psi_list[n])/inner(psi_list[n]',psi_list[n]))")
    end
end     
            

┌ Warning: Invariant subspace of dimension 2 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 2`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


length(e) = 4
site 1 : energy = [-25.059016994374947, -24.25, -23.940983005625053] linkdim = 2 trunc = 0.0
site 2 : energy = [-25.18969262078591, -24.07635182233307, -23.48395555688102] linkdim = 4 trunc = 0.0
site 3 : energy = [-25.437174543547968, -24.673268553508812, -24.04673790477002] linkdim = 6 trunc = 0.0
site 4 : energy = [-25.590852368142237, -25.345120866828665, -24.79474358414724] linkdim = 6 trunc = 0.0
site 5 : energy = [-25.817630109766675, -25.237273714862923, -24.67913604013701] linkdim = 6 trunc = 0.0
site 6 : energy = [-25.974345461500036, -25.346449565137373, -24.841560583230503] linkdim = 6 trunc = 0.0
site 7 : energy = [-26.180540142478566, -25.652206691259835, -25.16615916660208] linkdim = 6 trunc = 0.0
site 8 : energy = [-26.345793052640747, -25.788364664024314, -25.322995909704243] linkdim = 6 trunc = 0.0
site 9 : energy = [-26.533932513326395, -26.006782685262557, -25.563779617897005] linkdim = 6 trunc = 0.0
site 10 : energy = [-26.706488132372563, -26.1683714

site 83 : energy = [-39.741110040450884, -39.48019014636879, -39.16003873847476] linkdim = 6 trunc = 0.0
site 84 : energy = [-39.917870101964574, -39.65695058612041, -39.336798124032754] linkdim = 6 trunc = 0.0
site 85 : energy = [-40.094630170957736, -39.83371048204482, -39.51355898578757] linkdim = 6 trunc = 0.0
site 86 : energy = [-40.27139023395669, -40.01047077882995, -39.69031863368013] linkdim = 6 trunc = 0.0
site 87 : energy = [-40.448150302475554, -40.18723073904327, -39.86707919137115] linkdim = 6 trunc = 0.0
site 88 : energy = [-40.624910366463375, -40.36399094739912, -40.04383901047052] linkdim = 6 trunc = 0.0
site 89 : energy = [-40.80167043716568, -40.540750947353274, -40.22059937579734] linkdim = 6 trunc = 0.0
site 90 : energy = [-40.97843050215387, -40.71751110098316, -40.39735932228064] linkdim = 6 trunc = 0.0
site 91 : energy = [-41.15519058029752, -40.894271125518536, -40.57411955929434] linkdim = 6 trunc = 0.0
site 92 : energy = [-41.33195064701342, -41.071031245263

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


length(e) = 24
site 99 : energy = ComplexF64[-42.65716531260085 + 0.0im, -42.39624402518644 + 0.0im, -42.26499038662221 + 0.0im] linkdim = 2 trunc = 0.0
site 98 : energy = [-42.69268470962092, -42.431764673059156, -42.26794864335796] linkdim = 4 trunc = 0.0
site 97 : energy = [-42.697620972365854, -42.43670096536006, -42.41199624817647] linkdim = 8 trunc = 0.0
site 96 : energy = [-42.726692353693586, -42.46577272611418, -42.41428340132381] linkdim = 16 trunc = 0.0
site 95 : energy = [-42.734559910132376, -42.509995746744224, -42.47364030652191] linkdim = 20 trunc = 3.388022406860621e-16
site 94 : energy = [-42.76023720681093, -42.51293525083792, -42.499317724717] linkdim = 20 trunc = 5.97494363544829e-15
site 93 : energy = [-42.76991511691289, -42.58483755908205, -42.508995645878116] linkdim = 20 trunc = 1.1322365129042097e-14
site 92 : energy = [-42.79355338746533, -42.58881902104967, -42.53263394904937] linkdim = 20 trunc = 6.79787156947625e-14
site 91 : energy = [-42.804438072384976

...

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31


site 98 : energy = ComplexF64[-44.11763050270927 + 0.0im, -44.072108893886664 + 0.0im, -44.06882802359729 + 0.0im] linkdim = 4 trunc = 0.0
site 97 : energy = [-44.117737242395336, -44.072365448790926, -44.06939129233997] linkdim = 8 trunc = 0.0
site 96 : energy = [-44.11776247646699, -44.072409619034154, -44.069481483916] linkdim = 16 trunc = 0.0
site 95 : energy = [-44.11792398930083, -44.07277358134951, -44.07026609505199] linkdim = 20 trunc = 4.979737489514403e-9
site 94 : energy = [-44.11798289645395, -44.072871139543125, -44.070446150412465] linkdim = 20 trunc = 3.177982396794107e-8
site 93 : energy = [-44.118174281621975, -44.073272125627604, -44.071282751874065] linkdim = 20 trunc = 7.382574082515446e-8
site 92 : energy = [-44.11826447988234, -44.0734159100224, -44.07152446826189] linkdim = 20 trunc = 2.083466194041156e-7
site 91 : energy = [-44.11847218177773, -44.07382498321604, -44.07234225010386] linkdim = 20 trunc = 3.3635707434053864e-7
site 90 : energy = [-44.118589204730

site 31 : energy = [-44.12263192229534, -44.08090449295892, -44.080526871082455] linkdim = 20 trunc = 0.00014071023204691575
site 30 : energy = [-44.12246753243176, -44.08059741279128, -44.080217752910436] linkdim = 20 trunc = 0.00011506213521761905
site 29 : energy = [-44.12230874303634, -44.0804052112592, -44.079965289039286] linkdim = 20 trunc = 0.00015159117869408404
site 28 : energy = [-44.122119354889875, -44.08005243847766, -44.0796461100642] linkdim = 20 trunc = 0.00011294173349651948
site 27 : energy = [-44.12195754825534, -44.07983596158343, -44.079408889191996] linkdim = 20 trunc = 0.00016296735062764733
site 26 : energy = [-44.121742598808794, -44.079440211843355, -44.07906839758664] linkdim = 20 trunc = 0.00011397409671214955
site 25 : energy = [-44.12157466755531, -44.079201830995444, -44.07883261698395] linkdim = 20 trunc = 0.00017473271430588276
site 24 : energy = [-44.121334198226634, -44.07876298674577, -44.078467733593236] linkdim = 20 trunc = 0.00011554009352211363


┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\wqyan\.julia\packages\KrylovKit\diNbc\src\eigsolve\lanczos.jl:31
┌ Warning: Calling `inner(x::MPS, A::MPO, y::MPS)` where the site indices of the `MPS`
│ `x` and the `MPS` resulting from contracting `MPO` `A` with `MPS` `y` don't
│ match is deprecated as of ITensors v0.3 and will result in an error in ITensors
│ v0.4. The most common cause of this is something like the following:
│ 
│ ```julia
│ s = siteinds("S=1/2")
│ psi = randomMPS(s)
│ H = MPO(s, "Id")
│ inner(psi, H, psi)
│ ```
│ 
│ `psi` has the Index structure `-s-(psi)` and `H` has the Index structure
│ `-s'-(H)-s-`, so the Index structure of would be `(dag(psi)-s- -s'-(H)-s-(psi)`
│  unless the prime levels were fixed. Previously we tried fixing the prime level
│   in situations like this, but we will no longer be

The 0th excited state energy is: -44.11776696102699 + 0.0im
The 1th excited state energy is: -44.07214216024745 + 0.0im
The 2th excited state energy is: -44.06887348584615 + 0.0im
